# Add group label for test dataset

In [1]:
import pandas as pd

train_path = "data/Product10K/train.csv"
test_path = "data/Product10K/test_kaggletest.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
print(f"train: {train_df.shape}, test: {test_df.shape}")

train: (141931, 3), test: (55376, 3)


In [2]:
test_df.head()

,name,class,Usage
0,3963142.jpg,0,Public
1,1209854.jpg,0,Public
2,8638678.jpg,0,Private
3,6997109.jpg,0,Private
4,3134887.jpg,0,Private


In [3]:
class_group = {}

for i, row in train_df.iterrows():
    if row['class'] not in class_group:
        class_group[row['class']] = row['group']
    else:
        if class_group[row['class']] != row['group']:
            print(f"mismatched class group")

# class_group

In [4]:
test_df['group'] = test_df['class'].apply(lambda x: class_group[x])
test_df.head()

,name,class,Usage,group
0,3963142.jpg,0,Public,0
1,1209854.jpg,0,Public,0
2,8638678.jpg,0,Private,0
3,6997109.jpg,0,Private,0
4,3134887.jpg,0,Private,0


In [ ]:
# test_df_group_label = pd.read_csv("data/Product10K/test_group_label.csv")
# test_df_group_label.head()

In [ ]:
# group_label = {}

# for i, row in test_df_group_label.iterrows():
#     if row['group'] not in group_label:
#         group_label[row['group']] = row['flag'].replace('/', '、').replace('-', '、').strip()

# print(len(group_label))
# group_label

In [ ]:
# import re

# group_label_path = "data/Product10K/group_label.csv"
# group_label_df = pd.DataFrame(list(group_label.items()), columns=['group', 'flag'])
# group_label_df = group_label_df.sort_values('group')

# group_label_df.to_csv(group_label_path, index=False)

In [5]:
group_label_path = "data/Product10K/group_label.csv"
group_label_df = pd.read_csv(group_label_path)
# group_label_df['label'] = group_label_df['label'].apply(lambda x: x.replace('-', '、'))
# group_label_df.to_csv(group_label_path)
group_label_df.head()

,group,flag
0,0,服装类、男装、时尚服饰
1,1,服装类、外套、夹克
2,2,冬季服装、羽绒服
3,3,工装、工作服
4,4,儿童运动服装


In [6]:
test_df = pd.merge(test_df, group_label_df, on='group')
print(test_df.shape)
test_df.head()

(55376, 5)


,name,class,Usage,group,flag
0,3963142.jpg,0,Public,0,服装类、男装、时尚服饰
1,1209854.jpg,0,Public,0,服装类、男装、时尚服饰
2,8638678.jpg,0,Private,0,服装类、男装、时尚服饰
3,6997109.jpg,0,Private,0,服装类、男装、时尚服饰
4,3134887.jpg,0,Private,0,服装类、男装、时尚服饰


In [7]:
test_df = test_df.rename(columns={'name': 'filename', 'label': 'flag'})
test_df.head()

,filename,class,Usage,group,flag
0,3963142.jpg,0,Public,0,服装类、男装、时尚服饰
1,1209854.jpg,0,Public,0,服装类、男装、时尚服饰
2,8638678.jpg,0,Private,0,服装类、男装、时尚服饰
3,6997109.jpg,0,Private,0,服装类、男装、时尚服饰
4,3134887.jpg,0,Private,0,服装类、男装、时尚服饰


In [8]:
test_df.to_csv("data/Product10K/test_group_label.csv", index=False)

In [ ]:
# import boto3
# import json
# import base64
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import time
# import random

# # Initialize Bedrock client
# bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')

# def encode_image_to_base64(image_path):
#     """Convert image to base64 string"""
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# def call_claude_for_group(group_num, image_names):
#     """Call Claude 3.7 to get group label for a set of images"""
#     try:
#         # Prepare images for Claude
#         images_data = []
#         for img_name in image_names[:5]:  # Take only first 5 images
#             img_path = os.path.join("data/Product10K/test", img_name)
#             if os.path.exists(img_path):
#                 img_base64 = encode_image_to_base64(img_path)
#                 images_data.append({
#                     "type": "image",
#                     "source": {
#                         "type": "base64",
#                         "media_type": "image/jpeg",
#                         "data": img_base64
#                     }
#                 })
        
#         if not images_data:
#             return group_num, "No valid images found"
        
#         # Prepare the message content
#         content = [
#             {
#                 "type": "text",
#                 "text": "请看这些产品图片，提供一个简洁准确的细分商品类别标签，最好地描述这些是什么类型的产品。专注于主要的产品类别（例如：'电子产品'、'服装'、'家居用品'、'运动器材'等）。请只回答类别名称，不需要额外说明。"
#             }
#         ] + images_data
        
#         # Prepare the request
#         request_body = {
#             "anthropic_version": "bedrock-2023-05-31",
#             "max_tokens": 50,
#             "temperature": 0.1,
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": content
#                 }
#             ]
#         }
        
#         # Call Bedrock
#         response = bedrock_runtime.invoke_model(
#             modelId="anthropic.claude-3-5-sonnet-20241022-v2:0",
#             body=json.dumps(request_body)
#         )
        
#         # Parse response
#         response_body = json.loads(response['body'].read())
#         label = response_body['content'][0]['text'].strip()
        
#         print(f"Group {group_num}: {label}")
#         return group_num, label
        
#     except Exception as e:
#         print(f"Error processing group {group_num}: {str(e)}")
#         return group_num, f"Error: {str(e)}"

# # Get unique groups and sample images
# unique_groups = test_df['group'].unique()
# print(f"Total groups to process: {len(unique_groups)}")

# # Prepare data for concurrent processing
# group_data = []
# for group in unique_groups:
#     group_images = test_df[test_df['group'] == group]['name'].tolist()
#     # Randomly sample 5 images from the group
#     sampled_images = random.sample(group_images, min(5, len(group_images)))
#     group_data.append((group, sampled_images))

# # Process groups concurrently
# group_labels = {}
# max_workers = 5  # Limit concurrent requests to avoid rate limits

# print("Starting concurrent Claude API calls...")
# with ThreadPoolExecutor(max_workers=max_workers) as executor:
#     # Submit all tasks
#     future_to_group = {
#         executor.submit(call_claude_for_group, group, images): group 
#         for group, images in group_data
#     }
    
#     # Process completed tasks
#     for future in as_completed(future_to_group):
#         group_num, label = future.result()
#         group_labels[group_num] = label
        
#         # Add small delay to respect rate limits
#         time.sleep(0.1)

# print(f"\nCompleted processing {len(group_labels)} groups")
# print("\nGroup Labels:")
# for group, label in sorted(group_labels.items()):
#     print(f"Group {group}: {label}")


In [ ]:
# group_labels[5] = "服装 - 男装冬季服饰"
# group_labels[52] = "日用百货"
# group_labels[55] = "皮具配饰/钱包"
# group_labels[73] = "文具用品/钢笔"
# group_labels[100] = "打印机耗材"
# group_labels[136] = "手机配件"
# group_labels[187] = "饮品和食品类/早餐谷物"
# group_labels[194] = "玩具模型/玩具类"
# group_labels[197] = "儿童玩具/益智玩具"
# group_labels[201] = "工艺品/装饰品"
# group_labels[204] = "服饰配件/皮带"
# group_labels[210] = "服饰配件"
# group_labels[213] = "服饰配件/皮带"
# group_labels[215] = "儿童用品"
# group_labels[238] = "个人护理用品"
# group_labels[305] = "护理保健品"
# group_labels[329] = "家居用品/装饰品"
# group_labels[348] = "家居装饰品"
# group_labels[351] = "家居用品/晾衣架"
# group_labels[358] = "家居装饰品/花瓶"

In [ ]:
# import json
# import re

# group_label_path = "data/Product10K/group_label.csv"

# group_label_df = pd.DataFrame(list(group_labels.items()), columns=['group', 'label'])
# group_label_df = group_label_df.sort_values('group')
# for i, row in group_label_df.iterrows():
#     label = row['label']
#     label = re.sub(r' - ', '/', label)
#     group_label_df.loc[i, 'label'] = label.strip()

# group_label_df.to_csv(group_label_path, index=False)
# group_label_df.head()


In [ ]:
len(group_label_df['label'].unique())

In [ ]:
# df_merged = pd.merge(test_df, group_label_df, on='group')
# df_merged.head()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

# Specify group number
group_number = 50

# Filter images from the specified group
group_images = test_df[test_df['group'] == group_number]['name'].head(10).tolist()

# Create figure with 2 rows, 5 columns
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle(f'Group {group_number} Images', fontsize=16)

# Display images
for i, img_name in enumerate(group_images):
    row = i // 5
    col = i % 5
    
    # Load and display image
    img_path = os.path.join("data/Product10K/test", img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        axes[row, col].imshow(img)
        axes[row, col].set_title(img_name, fontsize=8)
        axes[row, col].axis('off')
    else:
        axes[row, col].text(0.5, 0.5, f'{img_name}\nNot Found', 
                           ha='center', va='center', transform=axes[row, col].transAxes)
        axes[row, col].axis('off')

# Hide empty subplots if less than 10 images
for i in range(len(group_images), 10):
    row = i // 5
    col = i % 5
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print(f"Showing {len(group_images)} images from group {group_number}, {group_label_df[group_label_df['group']==group_number]}")


# Prepare Nova SFT data

## Shein data set

In [ ]:
import pandas as pd
import json

# Constants
account_id = "452145973879"
s3_bucket = "sagemaker-us-east-1-452145973879"
# s3_prefix = "projects/nova-image-tagging/data"

system_prompt = """# Image Classification Assistant
    
    ## Role
    You are an image analyzer specialized in classifying visual content according to provided reference categories.
    
    ## Analysis Framework
    You must follow this 4-step process for every image:
    
    ### Step 1: Physical Attribute Extraction
    - Extract the objective physical attributes of all items in the image.
    - Focus strictly on neutral, geometric, and material properties.
    - **DO NOT** interpret the item's purpose, function, or potential use.
    - **Examples of attributes to extract:**
    -   **Shape:** Cylindrical, rectangular, pointed tip, curved edge, ring-shaped.
    -   **Components:** Handle, blade, trigger, nozzle, switch, USB port, chain.
    -   **Material/Texture:** Metallic, plastic, fabric, wooden, smooth, serrated.
    -   **Text/Markings:** Any visible letters, numbers, or symbols.
    
    ### Step 2: Feature Analysis
    Based on your description, identify key features:
    - Look for specific shapes (cylindrical, pointed, etc.)
    - Identify materials (metal, plastic, fabric, etc.)
    - Note functional elements (buttons, switches, blades, etc.)
    - Check for size indicators or child-oriented designs
    
    ### Step 3: Category Matching
    Compare identified features with the reference categories:
    - Match observed features with the "Key Features" listed for each category
    - Consider the "Definition" to understand the intended use or design purpose
    - Determine if the item meets the criteria for any category
    
    ### Step 4: Final Classification
    Provide classification result following these rules:
    - Return only the Chinese category names (the values in parentheses)
    - If multiple categories apply, separate them with English commas ","
    - Maximum of 5 categories
    - If no categories match, return "无"
    - Format your response as JSON: {"result":"target"}
    
    ## Output Requirements
    - Perform Steps 1-3 internally (do not show your analysis)
    - Only output the final JSON result from Step 4
    - Use Chinese category names only
    - Follow the exact JSON format specified
    
    ## Important Guidelines
    - Be precise - only classify items that clearly match the definitions
    - Consider both definition and key features when classifying
    - If uncertain about a match, err on the side of caution and exclude it
    - Do NOT include escaped Unicode characters in your output

    ## Reference Categories

    ### Knives and Blades
    1. **匕首 (Dagger)**  
       - **Definition**: A short-bladed weapon designed primarily for thrusting or stabbing attacks, characterized by a sharp pointed tip and compact design optimized for single-hand use.
       - **Key Features**: Sharp pointed tip (essential), short blade, designed for thrusting, single or double-edged, compact for close-quarters use.
    
    2. **刀具 (Knives)**  
       - **Definition**: Includes all types of knives, such as utility, tactical, folding, or ornamental blades, designed for cutting or slashing.  
       - **Key Features**: Sharp blade edges, handle designs (with or without finger rings), sheath or folding mechanisms, blade-like structures.

    3. **弹簧刀 (Switchblade)**  
       - **Definition**: A folding knife with a blade that deploys automatically via a spring mechanism, often triggered by a button.  
       - **Key Features**: Button or lever on handle, blade concealed in handle when folded, rapid deployment action implied by design.
    
    4. **蝴蝶刀 (Butterfly Knife)**  
       - **Definition**: A folding pocket knife with two handles that counter-rotate around the tang, allowing the blade to be concealed or deployed.  
       - **Key Features**: No curved handle, dual pivoting handles, blade sandwiched between handles when closed, often metallic with latch mechanism.
    
    5. **伪装刀具 (Disguised Knife)**  
       - **Definition**: Knives concealed within everyday objects, such as pens, credit cards, or belts, to evade detection.  
       - **Key Features**: Hidden blade mechanism, innocuous outer appearance (e.g., lipstick case, keychain), deployable sharp edge.
    
    6. **爪刀 (Claw Knife)**  
       - **Definition**: A knife designed for slashing or gripping, featuring a curved or slightly straight blade with one or more finger rings or holes in the handle for enhanced grip and hand-worn capability. Includes tactical, folding, and ornamental variations, distinguished by the presence of finger rings.  
       - **Key Features**: One or more finger holes or rings in the handle as the primary identifier, curved or slightly straight blade.
    
    ### Weapons and Restricted Items
    
    7. **仿真子弹 (Imitation Bullet)**  
       - **Definition**: Replica ammunition, often used in keychains or necklaces (e.g., bullet keychains, bullet necklaces).  
       - **Key Features**: Cylindrical casing with pointed tip, metallic sheen, no live components.
    
    8. **警用喷雾 (Police Spray)**  
       - **Definition**: Aerosol sprays including pepper, chili, tear gas, or anti-wolf/safety sprays.  
       - **Key Features**: Canister with nozzle, warning labels, handheld design.
    
    9. **警棍/甩棍 (Baton/Swing Stick)**  
       - **Definition**: Prohibited metal batons made of high-strength alloy steel, often with aggressive protrusions, some with police markings.  
       - **Key Features**: Telescoping sections, metallic finish, protrusions (e.g., spikes, barbs), police insignia.
    
    10. **电击器/电击棒 (Stun Gun/Stun Baton)**  
        - **Definition**: Devices with exposed high-voltage electrodes and aggressive protrusions, producing electric arcs.  
        - **Key Features**: Exposed electrodes, protrusions (e.g., spikes, barbs), visible electric arc or light.
    
    11. **指虎铁莲花 (Knuckle Duster/Iron Lotus)**  
        - **Definition**: Hand-worn offensive weapons including knuckle dusters, push swords, wolverine claws, iron lotus, finger rings, or key sticks.  
        - **Key Features**: Metal rings for fingers, protruding spikes or blades, fist-enclosing design.
    
    ### Explosive or Hazardous Items
    
    12. **卡式炉 (Cassette Stove)**  
        - **Definition**: A portable cooking stove using gas canisters.  
        - **Key Features**: Burner head, gas canister slot, foldable legs, ignition switch.
    
    13. **烟花爆竹 (Fireworks and Firecrackers)**  
        - **Definition**: A pyrotechnic product that produces bright sparks by igniting a chemical coating.
        - **Key Features**: Rod/cylinder; produces sparks and flame; requires ignition; often has a pointed base for insertion; typically covered in glitter/metallic finish.
    
    14. **射钉弹 (Nail Gun Cartridge)**  
        - **Definition**: Ammunition for nail guns, containing explosive charges to drive nails.  
        - **Key Features**: Cylindrical cartridge with metallic tip, labeled as ammunition, compatible with nail gun devices.
    
    15. **打火石、打火棒 (Flint or Fire Starter)**  
        - **Definition**: Tools using flint or metal rods to generate sparks for fire starting.  
        - **Key Features**: Metallic rod or stone, striker tool, spark-generating surface.
    
    16. **气罐 (Gas Canister)**  
        - **Definition**: Pressurized containers banned for cross-border transport, containing flammable or compressed gases.  
        - **Key Features**: Cylindrical metal container, pressure valve, hazard labels, sealed cap.
    
    17. **压罐喷雾 (Aerosol Spray Can)**  
        - **Definition**: Pressurized cans dispensing flammable or hazardous sprays.  
        - **Key Features**: Metal canister, nozzle with trigger, pressure release valve, warning labels.
    
    18. **烟雾弹 (Smoke Bomb)**  
        - **Definition**: Devices releasing colored smoke, often used for signaling or entertainment.  
        - **Key Features**: Small cylindrical or spherical shape, fuse or ignition point, smoke-emitting vent.
    
    19. **固体酒精 (Solid Alcohol)**  
        - **Definition**: Solid fuel blocks used for heating or cooking.  
        - **Key Features**: Rectangular or disc-shaped blocks, wax or gel-like texture, flammable packaging.
    
    20. **镁粉 (Magnesium Powder)**  
        - **Definition**: Fine magnesium powder used as a flammable material.  
        - **Key Features**: Silvery powder, airtight packaging, flammable warning labels.
    
    21. **烟饼（片） (Smoke Cake/Piece)**  
        - **Definition**: Compact smoke-producing devices, often for rituals or effects.  
        - **Key Features**: Flat or disc shape, ignition point, smoke-emitting surface.
    
    22. **重燃蜡烛 (Relight Candles)**  
        - **Definition**: Candles designed to relight after being blown out.  
        - **Key Features**: Waxed wick with embedded relighting mechanism, candle shape, flammable material.
    
    23. **火柴 (Matches)**  
        - **Definition**: Small sticks with a combustible head for ignition.  
        - **Key Features**: Wooden or paper sticks, colored ignitable tip, striking surface.
    
    24. **炸药 (Explosives)**  
        - **Definition**: High-explosive materials used for demolition or blasting.  
        - **Key Features**: Block or granular form, industrial packaging, explosive hazard symbols, detonation wiring.
    
    25. **红磷、白磷 (Red Phosphorus/White Phosphorus)**  
        - **Definition**: Highly reactive phosphorus compounds used in explosives or incendiaries.  
        - **Key Features**: Red or white crystalline powder, sealed containers, chemical hazard labels.
    
    26. **火药 (Gunpowder)**  
        - **Definition**: A combustible powder used as a propellant or explosive.  
        - **Key Features**: Granular or fine powder appearance, packaging with explosive warnings, metallic or plastic containers.
    
    27. **礼花筒 (Firework Tube)**  
        - **Definition**: A celebratory device that ejects confetti or streamers through mechanical action (non-combustible).
        - **Key Features**: Cylindrical tube; ejects paper/streamers; no fire or heat; often has twisting/pushing mechanism at base.
    
    ### Electronics and Appliances
    
    28. **充电宝 (Power Bank)**  
        - **Definition**: A portable device used to charge other electronic devices via a built-in battery.  
        - **Key Features**: Rectangular or cylindrical shape, USB ports (input/output), battery indicator lights, compact design.
    
    29. **纯锂电池 (Pure Lithium Battery)**  
        - **Definition**: Standalone lithium batteries, identifiable by CR/BR markings or labeled as "lithium battery/锂电池," excluding those sold with other products.  
        - **Key Features**: CR/BR prefix on labeling, cylindrical or coin-shaped, metallic casing, no accompanying devices.
    
    30. **电子打火器 (Electronic Lighter)**  
        - **Definition**: Lighters using piezoelectric crystals to generate high-voltage sparks, including rechargeable or battery-powered models.  
        - **Key Features**: Pressable crystal or button, electrode gap for sparks, charging port or battery compartment.
    
    31. **打火器（无燃气） (Gas-Free Lighter)**  
        - **Definition**: Lighters or electronic ignition devices without gas fuel, including flint or piezoelectric types.  
        - **Key Features**: Flint wheel or piezoelectric crystal, no visible gas tank, compact ignition design.
    
    32. **燃气打火机 (Gas Lighter)**  
        - **Definition**: Lighters powered by flammable gas (e.g., butane).  
        - **Key Features**: Gas tank visible, ignition trigger, flame adjustment knob, compact design.

    ### Children's Products
    33. **儿童包 (Children's Bag)**  
        - **Definition**: A bag designed specifically for children, varying in size and purpose, used for carrying personal items, school supplies, or toys.  
        - **Key Features**: Playful designs, cartoon characters, or themed motifs. Secure zipper closure. Various carrying options (wrist, handle, shoulder). Child-sized, with compartments or single storage, sometimes with charms.

    34. **童鞋 (Children's Shoes)**  
        - **Definition**: Shoes for children, indicated by text, images, or sizes (foot length ≤240mm).  
        - **Key Features**: Small size, child-friendly design, labeled size range.

    35. **婴童帽子 (Infant/Toddler Hat)**  
        - **Definition**: Headwear specifically designed for infants and toddlers, primarily for warmth or sun protection.  
        - **Key Features**: Soft, comfortable material, providing warmth or sun protection. Often features cartoon patterns, animal shapes, or cute prints. May include ear flaps, chin straps, or pom-pom decorations for comfort and secure fit.

    36. **儿童发饰 (Children's Hair Accessories)**  
        - **Definition**: Decorative hair clips, headbands, or hair ties specifically designed for children, used to secure or adorn hairstyles.  
        - **Key Features**: Brightly colored and varied in shape (e.g., bows, stars, butterflies), often featuring cartoon patterns or cute embellishments. Typically secured with clips, elastic bands, or ties, ensuring they are safe and easy to wear.

    37. **儿童太阳镜 (Children's Sunglasses)**  
        - **Definition**: Eyewear specifically designed for children to protect their eyes from the sun's harmful UV rays, often featuring playful designs.  
        - **Key Features**: UV protection, durable and child-friendly materials, various fun shapes (e.g., heart, flower, classic, cat-eye), colorful frames and lenses, and sometimes decorative elements like glitter or animal ears.

    38. **儿童自行车座椅 (Child Bicycle Seat)**  
        - **Definition**: A bicycle seat designed for children, indicated by text or images claiming child suitability.  
        - **Key Features**: Small seat attached to bike frame, child-sized design, safety straps or padding.

    ### Other Restricted Items
    39. **手铐、拇指烤、脚镣 (Handcuffs/Thumb Cuffs/Leg Irons)**  
        - **Definition**: Metal restraints meeting size criteria: handcuffs (>5cm inner diameter), thumb cuffs (>1cm), leg irons (>8cm).  
        - **Key Features**: Metal construction, hinged or linked design, size consistent with restraint purpose."""

# system_prompt = "请分析图片。"
user_prompt = "Please classify the item in this image according to the categories defined in the system."

def create_record(flag, s3_prefix, filename):
    s3_path = f"s3://{s3_bucket}/{s3_prefix}/{filename}"

    img_format = "jpeg" if filename.endswith(".jpg") else filename.split('.')[-1]

    if img_format not in ['jpeg']:
        # print(img_format)
        return None
    
    return {
        "schemaVersion": "bedrock-conversation-2024",
        "system": [{
            "text": system_prompt
        }],
        "messages": [{
                "role": "user",
                "content": [
                    {
                        "image": {
                            "format": img_format,
                            "source": {
                                "s3Location": {
                                    "uri": s3_path,
                                    "bucketOwner": account_id
                                }
                            }
                        }
                    },
                    {	"text": user_prompt	}
                ]
            },
            {
                "role": "assistant",
                "content": [{
                    "text": flag
                }]
            }
        ]
    }

### Test set

In [10]:
# Load data
df = pd.read_excel('data/shein_data/test_data.xlsx')

s3_prefix = "projects/nova-image-tagging/data/shein_data/images"
test_count = 0
# Generate JSONL
with open('data/merged_data/nova_sft_testset.jsonl', 'w', encoding='utf-8') as f:
    for idx, row in df.iterrows():
        gt_label = '{"result":"' + row['flag'] + '"}'

        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            test_count += 1


print(f"Generated {test_count} records in nova_sft_dataset.jsonl")

Generated 434 records in nova_sft_dataset.jsonl


### Train dataset

In [11]:
df = pd.read_excel('data/shein_data/train_data_balanced.xlsx')

s3_prefix = "projects/nova-image-tagging/data/shein_data/images"
train_count = 0
# Generate JSONL
with open('data/merged_data/nova_sft_trainset.jsonl', 'w', encoding='utf-8') as f:
    for idx, row in df.iterrows():
        gt_label = '{"result":"' + row['flag'] + '"}'

        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            train_count += 1

print(f"Generated {train_count} records in nova_sft_dataset.jsonl")

Generated 1418 records in nova_sft_dataset.jsonl


### Using Product10K dataset

In [12]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Your existing code
total_train = 20000
total_test = 1000

train_samples = total_train - train_count
test_samples = total_test - test_count 
total_samples = train_samples + test_samples
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def hybrid_stratified_sampling(test_df, total_samples, train_samples, test_samples, 
                             stratify_column='label', random_state=42):
    """
    Hybrid approach: stratified for large classes, random for small classes
    Keeps all classes including those with only 1 sample
    """
    
    print("=== Hybrid Stratified Sampling ===")
    print(f"Original data shape: {test_df.shape}")
    print(f"Target total samples: {total_samples}")
    print(f"Target train samples: {train_samples}")
    print(f"Target test samples: {test_samples}")
    
    # Analyze class distribution
    class_counts = test_df[stratify_column].value_counts().sort_values(ascending=False)
    small_classes = class_counts[class_counts == 1].index
    large_classes = class_counts[class_counts >= 2].index
    
    print(f"\n=== Class Analysis ===")
    print(f"Total classes: {len(class_counts)}")
    print(f"Small classes (1 sample): {len(small_classes)}")
    print(f"Large classes (>=2 samples): {len(large_classes)}")
    
    if len(small_classes) > 0:
        print(f"Small classes: {list(small_classes)}")
    
    # Show class distribution
    print(f"\n=== Class Distribution ===")
    for cls, count in class_counts.head(10).items():  # Show top 10
        print(f"  {cls}: {count} samples")
    if len(class_counts) > 10:
        print(f"  ... and {len(class_counts) - 10} more classes")
    
    # Handle the special case where all classes are small
    if len(large_classes) == 0:
        print("\n=== All Classes Are Small - Using Pure Random Sampling ===")
        return pure_random_sampling(test_df, total_samples, train_samples, test_samples, random_state)
    
    # Split data into small and large class groups
    small_class_df = test_df[test_df[stratify_column].isin(small_classes)]
    large_class_df = test_df[test_df[stratify_column].isin(large_classes)]
    
    print(f"\n=== Data Split ===")
    print(f"Small class samples: {len(small_class_df)}")
    print(f"Large class samples: {len(large_class_df)}")
    
    # Calculate sampling proportions
    total_original = len(test_df)
    small_class_ratio = len(small_class_df) / total_original
    large_class_ratio = len(large_class_df) / total_original
    
    print(f"Small class ratio: {small_class_ratio:.3f}")
    print(f"Large class ratio: {large_class_ratio:.3f}")
    
    # Calculate target samples for each group
    target_samples_from_small = int(total_samples * small_class_ratio)
    target_samples_from_large = total_samples - target_samples_from_small
    
    # Ensure we don't exceed available samples
    actual_samples_from_small = min(target_samples_from_small, len(small_class_df))
    actual_samples_from_large = min(target_samples_from_large, len(large_class_df))
    
    # Adjust if we can't get enough samples
    actual_total_samples = actual_samples_from_small + actual_samples_from_large
    if actual_total_samples < total_samples:
        print(f"Warning: Can only sample {actual_total_samples} instead of {total_samples}")
        # Try to compensate by taking more from the larger group
        if actual_samples_from_large < len(large_class_df):
            compensation = min(total_samples - actual_total_samples, 
                             len(large_class_df) - actual_samples_from_large)
            actual_samples_from_large += compensation
            actual_total_samples += compensation
    
    print(f"\n=== Sampling Plan ===")
    print(f"From small classes: {actual_samples_from_small} samples")
    print(f"From large classes: {actual_samples_from_large} samples")
    print(f"Total: {actual_samples_from_small + actual_samples_from_large} samples")
    
    # Sample from small classes (randomly, since we can't stratify single samples)
    sampled_small = pd.DataFrame()
    if actual_samples_from_small > 0 and len(small_class_df) > 0:
        print(f"\nSampling {actual_samples_from_small} from small classes (random sampling)...")
        sampled_small = small_class_df.sample(n=actual_samples_from_small, random_state=random_state)
        print(f"Small classes sampled: {len(sampled_small)} samples")
        small_class_distribution = sampled_small[stratify_column].value_counts()
        print("Small classes in sample:", dict(small_class_distribution))
    
    # Sample from large classes (with stratification)
    sampled_large = pd.DataFrame()
    if actual_samples_from_large > 0 and len(large_class_df) > 0:
        print(f"\nSampling {actual_samples_from_large} from large classes (stratified sampling)...")
        if actual_samples_from_large >= len(large_class_df):
            sampled_large = large_class_df.copy()
            print(f"Taking all {len(sampled_large)} samples from large classes")
        else:
            try:
                sampled_large, _ = train_test_split(
                    large_class_df,
                    train_size=actual_samples_from_large,
                    stratify=large_class_df[stratify_column],
                    random_state=random_state
                )
                print(f"Large classes sampled: {len(sampled_large)} samples (stratified)")
            except ValueError as e:
                print(f"Stratified sampling failed for large classes: {e}")
                print("Falling back to random sampling for large classes...")
                sampled_large = large_class_df.sample(n=actual_samples_from_large, random_state=random_state)
                print(f"Large classes sampled: {len(sampled_large)} samples (random)")
    
    # Combine samples
    sample_pieces = [df for df in [sampled_small, sampled_large] if not df.empty]
    
    if not sample_pieces:
        raise ValueError("No samples were selected!")
    
    sampled_df = pd.concat(sample_pieces, ignore_index=True)
    
    # Shuffle the combined dataset
    sampled_df = sampled_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    print(f"\n=== Sampling Results ===")
    print(f"Total sampled: {len(sampled_df)} samples")
    
    # Show final class distribution in sampled data
    final_class_counts = sampled_df[stratify_column].value_counts()
    print(f"Classes in sampled data: {len(final_class_counts)}")
    
    # Adjust train/test split sizes if needed
    actual_total = len(sampled_df)
    if train_samples + test_samples > actual_total:
        print(f"\nAdjusting train/test split (requested {train_samples + test_samples}, have {actual_total})")
        ratio = train_samples / (train_samples + test_samples)
        train_samples = int(actual_total * ratio)
        test_samples = actual_total - train_samples
        print(f"Adjusted: train={train_samples}, test={test_samples}")
    
    # Determine if we can use stratified split for train/test
    final_class_counts = sampled_df[stratify_column].value_counts()
    classes_too_small = final_class_counts[final_class_counts < 2]
    can_stratify = len(classes_too_small) == 0
    
    print(f"\n=== Train/Test Split ===")
    if can_stratify:
        print("Using stratified split for train/test")
        try:
            train_df, test_df_final = train_test_split(
                sampled_df,
                train_size=train_samples,
                test_size=test_samples,
                stratify=sampled_df[stratify_column],
                random_state=random_state
            )
        except ValueError as e:
            print(f"Stratified split failed: {e}")
            print("Falling back to random split")
            can_stratify = False
    
    if not can_stratify:
        print(f"Using random split for train/test ({len(classes_too_small)} classes have <2 samples)")
        if len(classes_too_small) > 0:
            print(f"Classes with <2 samples: {list(classes_too_small.index)}")
        
        train_df, test_df_final = train_test_split(
            sampled_df,
            train_size=train_samples,
            test_size=test_samples,
            random_state=random_state
        )
    
    print(f"\n=== Final Results ===")
    print(f"Train set: {len(train_df)} samples")
    print(f"Test set: {len(test_df_final)} samples")
    print(f"Total: {len(train_df) + len(test_df_final)} samples")
    
    return train_df, test_df_final, sampled_df

def pure_random_sampling(test_df, total_samples, train_samples, test_samples, random_state=42):
    """
    Fallback function for when all classes are too small for stratification
    """
    print("Using pure random sampling (no stratification possible)")
    
    # First sample
    if total_samples >= len(test_df):
        sampled_df = test_df.copy()
    else:
        sampled_df = test_df.sample(n=total_samples, random_state=random_state)
    
    # Adjust split sizes if needed
    actual_total = len(sampled_df)
    if train_samples + test_samples > actual_total:
        ratio = train_samples / (train_samples + test_samples)
        train_samples = int(actual_total * ratio)
        test_samples = actual_total - train_samples
    
    # Split
    train_df, test_df_final = train_test_split(
        sampled_df,
        train_size=train_samples,
        test_size=test_samples,
        random_state=random_state
    )
    
    return train_df, test_df_final, sampled_df

def validate_results(original_df, sampled_df, train_df, test_df_final, stratify_column='label'):
    """
    Validate and report on the sampling results
    """
    print("\n" + "="*50)
    print("VALIDATION REPORT")
    print("="*50)
    
    # Basic counts
    print(f"Original data: {len(original_df)} samples, {original_df[stratify_column].nunique()} classes")
    print(f"Sampled data: {len(sampled_df)} samples, {sampled_df[stratify_column].nunique()} classes")
    print(f"Train set: {len(train_df)} samples, {train_df[stratify_column].nunique()} classes")
    print(f"Test set: {len(test_df_final)} samples, {test_df_final[stratify_column].nunique()} classes")
    
    # Class distribution comparison
    orig_dist = original_df[stratify_column].value_counts(normalize=True).sort_index()
    sampled_dist = sampled_df[stratify_column].value_counts(normalize=True).sort_index()
    
    print(f"\n=== Class Distribution Comparison (Top 10) ===")
    comparison_df = pd.DataFrame({
        'Original_Count': original_df[stratify_column].value_counts(),
        'Original_Prop': orig_dist,
        'Sampled_Count': sampled_df[stratify_column].value_counts(),
        'Sampled_Prop': sampled_dist
    }).fillna(0)
    
    comparison_df['Prop_Diff'] = abs(comparison_df['Original_Prop'] - comparison_df['Sampled_Prop'])
    comparison_df = comparison_df.sort_values('Original_Count', ascending=False)
    
    print(comparison_df.head(10).round(4))
    
    # Check for large deviations
    large_deviations = comparison_df[comparison_df['Prop_Diff'] > 0.05]
    if len(large_deviations) > 0:
        print(f"\n⚠️  Warning: {len(large_deviations)} classes have >5% proportion deviation")
        print(large_deviations[['Original_Prop', 'Sampled_Prop', 'Prop_Diff']].head())
    else:
        print("\n✅ All class proportions are well preserved (<5% deviation)")
    
    # Check for missing classes
    missing_classes = set(orig_dist.index) - set(sampled_dist.index)
    if missing_classes:
        print(f"\n⚠️  Warning: {len(missing_classes)} classes missing from sample: {list(missing_classes)[:5]}...")
    else:
        print("\n✅ All classes preserved in sample")

# ============================================================================
# MAIN EXECUTION CODE
# ============================================================================

# Your original parameters
total_train = 20000
total_test = 1000

train_samples = total_train - train_count
test_samples = total_test - test_count 
total_samples = train_samples + test_samples

print("CONFIGURATION:")
print(f"train_count: {train_count}")
print(f"test_count: {test_count}")
print(f"train_samples needed: {train_samples}")
print(f"test_samples needed: {test_samples}")
print(f"total_samples needed: {total_samples}")

# Execute the hybrid stratified sampling
try:
    train_df_sample, test_df_sample, sampled_df = hybrid_stratified_sampling(
        test_df, 
        total_samples, 
        train_samples, 
        test_samples,
        stratify_column='flag',
        random_state=42
    )
    
    # Validate results
    validate_results(test_df, sampled_df, train_df_sample, test_df_sample, 'flag')
    
    print(f"\n🎉 SUCCESS!")
    print(f"Final train set: {len(train_df_sample)} samples")
    print(f"Final test set: {len(test_df_sample)} samples")
    
    # Show final class distributions
    print(f"\nTrain set class distribution:")
    print(train_df_sample['flag'].value_counts().head())
    
    print(f"\nTest set class distribution:")
    print(test_df_sample['flag'].value_counts().head())
    
except Exception as e:
    print(f"❌ Error occurred: {e}")
    import traceback
    traceback.print_exc()


CONFIGURATION:
train_count: 1418
test_count: 434
train_samples needed: 18582
test_samples needed: 566
total_samples needed: 19148
=== Hybrid Stratified Sampling ===
Original data shape: (55376, 5)
Target total samples: 19148
Target train samples: 18582
Target test samples: 566

=== Class Analysis ===
Total classes: 295
Small classes (1 sample): 0
Large classes (>=2 samples): 295

=== Class Distribution ===
  运动服装: 2326 samples
  奢侈品箱包: 2053 samples
  皮具配饰、钱包: 1699 samples
  休闲服装: 1653 samples
  运动休闲裤: 1438 samples
  玩具模型: 1039 samples
  儿童玩具、轨道玩具: 1031 samples
  儿童玩具、教育玩具: 1014 samples
  玩具车辆: 980 samples
  儿童玩具、益智玩具: 935 samples
  ... and 285 more classes

=== Data Split ===
Small class samples: 0
Large class samples: 55376
Small class ratio: 0.000
Large class ratio: 1.000

=== Sampling Plan ===
From small classes: 0 samples
From large classes: 19148 samples
Total: 19148 samples

Sampling 19148 from large classes (stratified sampling)...
Large classes sampled: 19148 samples (stratifie

In [13]:
train_df_sample.head()

,filename,class,Usage,group,flag
13627,3728212.jpg,1859,Private,37,运动鞋、洗涤用品
11986,4867774.jpg,2231,Private,43,劳保鞋
17341,7227083.jpg,9054,Private,344,家居用品、床上用品
17753,3394203.jpg,8278,Public,251,香水、香氛产品
409,2562261.jpg,8695,Private,291,化妆品、美妆用品


In [14]:
s3_prefix = "projects/nova-image-tagging/data/Product10K/test"

# Generate JSONL
with open('data/merged_data/nova_sft_testset.jsonl', 'a', encoding='utf-8') as f:
    for idx, row in test_df_sample.iterrows():
        gt_label = '{"result":"' + row['flag'] + '"}'

        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            test_count += 1


print(f"Generated {test_count} records in nova_sft_dataset.jsonl")

Generated 1000 records in nova_sft_dataset.jsonl


In [15]:
s3_prefix = "projects/nova-image-tagging/data/Product10K/test"

# Generate JSONL
with open('data/merged_data/nova_sft_trainset.jsonl', 'a', encoding='utf-8') as f:
    for idx, row in train_df_sample.iterrows():
        gt_label = '{"result":"' + row['flag'] + '"}'

        record = create_record(gt_label, s3_prefix, row['filename'])
        if record:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
            train_count += 1


print(f"Generated {train_count} records in nova_sft_dataset.jsonl")

Generated 20000 records in nova_sft_dataset.jsonl


In [1]:
## Read the file, replace text, write back
with open('data/merged_data/nova_sft_trainset.jsonl', 'r') as f:
    content = f.read()

content = content.replace('us-west-2', 'us-east-1')

with open('data/merged_data/nova_sft_trainset.jsonl', 'w') as f:
    f.write(content)

print("Done!")

Done!


In [10]:
import json

with open('data/merged_data/nova_sft_trainset.jsonl', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1: 
            break
        if line.strip():
            line = json.loads(line.strip())
            print(json.dumps(line, indent=2, ensure_ascii=False))

{
  "schemaVersion": "bedrock-conversation-2024",
  "system": [
    {
      "text": "# Image Classification Assistant\n\n    ## Role\n    You are an image analyzer specialized in classifying visual content according to provided reference categories.\n\n    ## Analysis Framework\n    You must follow this 4-step process for every image:\n\n    ### Step 1: Physical Attribute Extraction\n    - Extract the objective physical attributes of all items in the image.\n    - Focus strictly on neutral, geometric, and material properties.\n    - **DO NOT** interpret the item's purpose, function, or potential use.\n    - **Examples of attributes to extract:**\n    -   **Shape:** Cylindrical, rectangular, pointed tip, curved edge, ring-shaped.\n    -   **Components:** Handle, blade, trigger, nozzle, switch, USB port, chain.\n    -   **Material/Texture:** Metallic, plastic, fabric, wooden, smooth, serrated.\n    -   **Text/Markings:** Any visible letters, numbers, or symbols.\n\n    ### Step 2: Featur

In [8]:
import random

# Set random seed
random.seed(42)

# Read all lines
with open('data/merged_data/nova_sft_trainset.jsonl', 'r') as f:
    lines = f.readlines()

# Shuffle the lines
random.shuffle(lines)

# Write back
with open('data/merged_data/nova_sft_trainset.jsonl', 'w') as f:
    f.writelines(lines)

print("Done!")

Done!
